# TabM

This is a standalone usage example for the TabM project.
The easiest way to run it is [Pixi](https://pixi.sh/latest/#installation):

```shell
git clone https://github.com/yandex-research/tabm
cd tabm

# With GPU:
pixi run -e cuda jupyter-lab example.ipynb

# Without GPU:
pixi run jupyter-lab example.ipynb
```

For the full overview of the project, and for non-Pixi environment setups, see README in the repository:
https://github.com/yandex-research/tabm

This notebook is based on the original example: https://github.com/yandex-research/tabm/blob/main/example.ipynb

In [ ]:
!git clone https://github.com/yandex-research/tabm
!pip install wldhx.yadisk-direct rtdl_num_embeddings

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import math
import random
import warnings

import numpy as np
import pandas as pd
import rtdl_num_embeddings  # https://github.com/yandex-research/rtdl-num-embeddings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.auto import tqdm
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OrdinalEncoder

warnings.simplefilter('ignore')
from tabm_reference import Model, make_parameter_groups

In [ ]:
seed = 0
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
pass

### PyTorch settings

In [ ]:
# Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Automatic mixed precision (AMP)
# torch.float16 is implemented for completeness,
# but it was not tested in the project,
# so torch.bfloat16 is used by default.
amp_dtype = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float16
    if torch.cuda.is_available()
    else None
)
# Changing False to True will result in faster training on compatible hardware.
amp_enabled = False and amp_dtype is not None
grad_scaler = torch.cuda.amp.GradScaler() if amp_dtype is torch.float16 else None  # type: ignore

# torch.compile
compile_model = False

# fmt: off
print(
    f'Device:        {device.type.upper()}'
    f'\nAMP:           {amp_enabled} (dtype: {amp_dtype})'
    f'\ntorch.compile: {compile_model}'
)
# fmt: on

### Dataset
Dataset is "Regression with an Insurance Dataset"

https://www.kaggle.com/competitions/playground-series-s4e12/overview

if you have a kaggle account, you can download the data using kaggle API:

```python
kaggle competitions download -c playground-series-s4e12
```

In [ ]:
! curl -L $(yadisk-direct https://disk.yandex.ru/d/YbkU_KfAqGtdXg) -o insurance_dataset_train.csv
! curl -L $(yadisk-direct https://disk.yandex.ru/d/_gs3p3yvp0TNRg) -o insurance_dataset_test.csv

### Data Preprocessing

In [ ]:
# Upload train and test parts
train_df = pd.read_csv("./insurance_dataset_train.csv")
test_df = pd.read_csv("./insurance_dataset_test.csv")

Define lists with numerical and categorical column names.

Also we exclude `id` and `Policy Start Date` cols.

In [ ]:
target_col = "Premium Amount"

num_cols = ['Age', 'Annual Income', 'Number of Dependents', 'Health Score', 
            'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration']
cat_cols = ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
            'Policy Type', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
            'Property Type']

Split train part on train and val, test parts

In [ ]:
# If training is TOO slow, set to True
DEMO_MODE = True

In [ ]:
# collect number of unique values for categorical columns. Will be needed in future.
cat_cardinalities = train_df[cat_cols].nunique(dropna=False).to_list()

if DEMO_MODE:
    idx = np.random.randint(0, len(train_df), size=50000)
    train_df = train_df.iloc[idx]

all_idx = np.arange(len(train_df))
# Select the test part
train_idx, val_idx = train_test_split(all_idx, train_size=0.8)

# Fill dict with all parts we have
_train_df = train_df.iloc[train_idx]
_val_df = train_df.iloc[val_idx]

data = {
    'train': {
        'x_num': _train_df[num_cols],
        'x_cat': _train_df[cat_cols],
        'y': _train_df[target_col].to_numpy().astype(np.float32)
    },
    'val': {
        'x_num': _val_df[num_cols],
        'x_cat': _val_df[cat_cols],
        'y': _val_df[target_col].to_numpy().astype(np.float32)
    }
}

# Also process test data, we'll need it in the end
test_data = {
    "x_num": test_df[num_cols],
    "x_cat": test_df[cat_cols],
    'id': test_df['id'].to_numpy()
}

Define processing pipelines:

In [ ]:
n_quantiles = max(min(len(train_idx) // 30, 1000), 10)

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", QuantileTransformer(n_quantiles=n_quantiles, 
                                   output_distribution='normal',
                                   subsample=10**9))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OrdinalEncoder())
])

Apply processing operations to the data

In [ ]:
# The noise is added to improve the output of QuantileTransformer in some cases
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-5, data['train']['x_num'].shape)
)

# Fit pipelines on train part
num_processor = num_pipeline.fit(data['train']['x_num'] + noise)
cat_processor = cat_pipeline.fit(data['train']['x_cat'])

# Apply the processing to all parts. Note, that when processor is applied
# each dataframe converts to np.ndarray. Check that x_num array has float32 dtype
# and x_cat is int64.

### YOUR CODE HERE

print(f"`x_num` dtype: {type(data['train']['x_num'])}\n`x_cat` dtype: {type(data['train']['x_cat'])}")

# Apply processing to test data as well

### YOUR CODE HERE


Convert every part of the dataset to `torch.tensor`

In [ ]:
# Convert data to tensors
data_torch = {
    part: {k: torch.as_tensor(v, device=device) for k, v in data[part].items()}
    for part in data
}

test_data_torch = {
    "x_num": torch.as_tensor(test_data["x_num"], device=device),
    "x_cat": torch.as_tensor(test_data["x_cat"], device=device)
}

### TabM model

In [ ]:
# Choose one of the two configurations below.

# TabM or TabM-mini
# arch_type = 'tabm'
arch_type = 'tabm-mini'

# Use the piecewise-linear embeddings
bins = rtdl_num_embeddings.compute_bins(data_torch['train']['x_num'])
# bins = None

model = Model(
    n_num_features=len(num_cols),
    cat_cardinalities=cat_cardinalities,
    n_classes=None,
    backbone={
        'type': 'MLP',
        'n_blocks': 3 if bins is None else 2,
        'd_block': 512,
        'dropout': 0.1,
    },
    bins=bins,
    num_embeddings=(
        None
        if bins is None
        else {
            'type': 'PiecewiseLinearEmbeddings',
            'd_embedding': 16,
            'activation': False,
            'version': 'B',
        }
    ),
    arch_type=arch_type,
    k=32,
).to(device)

if compile_model:
    # NOTE
    # `torch.compile` is intentionally called without the `mode` argument
    # (mode="reduce-overhead" caused issues during training with torch==2.0.1).
    model = torch.compile(model)
    evaluation_mode = torch.no_grad
else:
    evaluation_mode = torch.inference_mode

In [ ]:
print("Number of params: ", sum(p.numel() for p in model.parameters()))
model

Since Loss in kaggle competition is RMSLE (Root Mean Squared Logarithmic Error) let's use it as validation metric

In [ ]:
from sklearn.metrics import root_mean_squared_log_error
def compute_rmsle(y_pred, y_true):
    y_pred = y_pred.detach().cpu().numpy()
    y_true = y_true.detach().cpu().numpy()
    return root_mean_squared_log_error(y_true, y_pred)

Let's define class for training and evaluating our model

In [ ]:
class Runner:
    """Runner for train/evaludate and predict using provided model."""
    def __init__(self, model, optimizer, loss, device, eval_metric=None, grad_scaler=None, 
                 epoch_bar=False, checkpoint_name="tabm_model.ckpt"):
        self.model = model
        self.optimizer = optimizer
        self.loss = loss
        self.device = device
        self.eval_metric = loss if eval_metric is None else eval_metric
        self.grad_scaler = grad_scaler
        self.epoch_bar = epoch_bar
        self.checkpoint_name = checkpoint_name

        self._train_mean = None
        self._train_std = None

    def compute_loss(self, y_pred, y_true):
        # TabM produces k predictions per object. Each of them must be trained separately.
        # (regression)     y_pred.shape == (batch_size, k)
        k = y_pred.shape[-1]
        return self.loss(y_pred.flatten(0, 1), y_true.repeat_interleave(k))

    def forward(self, batch_num, batch_cat=None, model=None):
        model = self.model if model is None else model
        return (
            model(batch_num, batch_cat)
            .squeeze(-1)
            .float()
        )

    def _train_step(self, y_pred, y_true, normalize_target):
        # We need to normalize target if needed and compute loss

        ### YOUR CODE HERE

        if self.grad_scaler is None:
            loss.backward()
            self.optimizer.step()
        else:
            self.grad_scaler.scale(loss).backward()
            self.grad_scaler.step(optimizer)
            self.grad_scaler.update()
        self.optimizer.zero_grad()
        return loss.detach().cpu().numpy().mean()
    
    def _eval_step(self, y_pred, y_true):
        # Compute the mean of the k predictions and scale prediction 
        # into it's original range

        y_pred = ### YOUR CODE HERE

        # Avoid computing gradients if train_phase is False
        with torch.set_grad_enabled(False):
            metric = self.eval_metric(y_pred, y_true)
        if isinstance(metric, torch.Tensor):
            return metric.detach().cpu().numpy()
        return metric

    def _run_epoch(self, data, batch_indices, train_phase=True, normalize_target=True, model=None):
        self.model.train(train_phase)
        epoch_history = []
        for batch_idx in tqdm(batch_indices, disable=not self.epoch_bar, leave=False):
            batch_num = data['x_num'][batch_idx]
            batch_cat = data['x_cat'][batch_idx]
            y_true = data['y'][batch_idx]
            y_pred = self.forward(batch_num, batch_cat, model=model)

            if train_phase:
                res = self._train_step(y_pred, y_true, normalize_target=normalize_target)
            else:
                res = self._eval_step(y_pred, y_true)
            epoch_history.append(res)
        return np.mean(epoch_history)

    def train(self, train_data, val_data, batch_size=256, n_epochs=1000, patience=16,
              eval_batch_size=None, normalize_target=True, save_best_model=False, 
              force_collect_stats=False):
        best = {
            'val': math.inf,
            'test': math.inf,
            'epoch': -1,
        }

        # Important!
        # For regression tasks it is highly recommended to standardize the training labels.
        self._train_mean = (train_data['y'].mean() 
                            if self._train_mean is None or force_collect_stats 
                            else self._train_mean)
        self._train_std = (train_data['y'].std() 
                           if self._train_std is None or force_collect_stats 
                           else self._train_std)

        # Early stopping: the training stops when
        # there are more than `patience` consequtive bad updates.
        remaining_patience = patience
        eval_batch_size = batch_size if eval_batch_size is None else eval_batch_size
        for epoch in tqdm(range(n_epochs), total=n_epochs):
            # Generate batch indices
            batch_indices = torch.randperm(len(train_data['y']), device=device).split(batch_size)
            # Perform train epoch
            train_score = self._run_epoch(train_data, batch_indices, train_phase=True,
                                          normalize_target=normalize_target)
            # Validate model after train epoch
            val_score = self.evaluate(val_data, eval_batch_size)

            msg = f"Epoch: {epoch} (train) {train_score:.4f} (val) {val_score:.4f}"
        
            if val_score < best['val']:
                best = {'val': val_score, 'epoch': epoch}
                remaining_patience = patience
                print(msg + "\t🌸 New best epoch! 🌸")
                if save_best_model:
                    torch.save(self.model, open(self.checkpoint_name, 'wb'))
            else:
                remaining_patience -= 1
                print(msg)
        
            if remaining_patience < 0:
                break
        print(f"\nTraining is complete.\nBest model: epoch {best['epoch']} metric {best['val']}")

    @evaluation_mode
    def evaluate(self, data, batch_size=256, model=None):
        batch_indices = torch.randperm(len(data['y']), device=device).split(batch_size)
        return self._run_epoch(data, batch_indices, train_phase=False, model=model)

    @evaluation_mode
    def predict(self, data, batch_size=256, model=None):
        batch_indices = torch.arange(len(data['x_num']), device=device).split(batch_size)
        preds = []
        for batch_idx in tqdm(batch_indices, disable=not self.epoch_bar, leave=False):
            batch_num = data['x_num'][batch_idx]
            batch_cat = data['x_cat'][batch_idx]
            # You need to make prediction for batch and store result in `preds`
            # Don't forget to scale prediction into it's orginal range
            y_pred = ### YOUR CODE HERE

            preds.extend(y_pred.detach().cpu().numpy())

        return preds

### Create runner instance and train the model

In [ ]:
optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)
loss = F.mse_loss
eval_metric = compute_rmsle

runner = Runner(model, optimizer, loss, device=device, eval_metric=eval_metric,
                grad_scaler=grad_scaler, epoch_bar=True)

Train model

In [ ]:
runner.train(data_torch['train'], data_torch['val'], batch_size=1024, n_epochs=15, save_best_model=False)

### How to improve the result??
* Play with network params
* Try to use or dont use embeddings
* Try different models (tabM, tabM_mini)
* (*) Tune model params using optuna ([source](https://optuna.org/), [examples](https://github.com/optuna/optuna-examples))
* Any other ideas?

### What about standard approaches? Let's try catboost!

In [ ]:
# if using colab
#!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

kwargs = {"task_type": 'GPU', "devices": "0"} if torch.cuda.is_available() else {}
catboost_model = CatBoostRegressor(**kwargs, iterations=500)

X_cb = train_df[cat_cols + num_cols].fillna(0)
y_cb = train_df[target_col]
catboost_model.fit(X_cb, y_cb, cat_features=cat_cols, verbose=50)

In [ ]:
catboost_preds = catboost_model.predict(test_df[cat_cols + num_cols].fillna(0))

### Make submission

Tips:
* Save your local "best" results, to prevent overfitting on validation part
* (simple) Before making submission (after params tuning), train model using whole train part (insurance_dataset_train)
* (harder) Make submission as an ensemble of cross-validated models on the train part of the dataset
* 

In [ ]:
def make_submission(preds, ids, path=None):
    result = pd.DataFrame({"id": ids, "Premium Amount": preds})
    if path is None:
        return result
    result.to_csv(path, index=False)

In [ ]:
# Upload model from checkpoint
# with open("tabm_model.ckpt", 'rb') as f:
#     best_model = torch.load(f)
# preds = runner.predict(test_data_torch, batch_size=2048, model=best_model)

In [ ]:
# Use model from last epoch
preds = runner.predict(test_data_torch, batch_size=2048)

In [ ]:
make_submission(preds, test_data["id"], "simple_submission.csv")

In [ ]:
make_submission(catboost_preds, test_df["id"], "simple_catboost_submission.csv")

### Submit using API

You can submit your solution directly from colab!


```python
kaggle competitions submit -c playground-series-s4e12 -f submission.csv -m "Message"
```